In [ ]:
if(grepl("kaggle",getwd())){
    paths <- list(
        data = file.path("/kaggle", "input", "gfi-iran"),
        output = file.path("/kaggle", "working","output"),
        temp = file.path("/kaggle", "working","intermediate-data")
    )
}else{
    paths <- list(
        data = file.path("..","data"),
        output = file.path("..","output"),
        temp = file.path("..","intermediate-data")
    )
    for(i in list.files(paths$data,full.names = TRUE)){
        unzip(i,exdir = paths$data)
    }
    file.rename(paths$data |> file.path("IRN_2017_FINDEX_v02_M_EXCEL\\micro_irn_varlabel.xls"),
    paths$data |> file.path("micro_irn_varlabel.xls"))
    file.rename(paths$data |> file.path("IRN_2017_FINDEX_v02_M_EXCEL\\micro_irn_varname.xls"),
    paths$data |> file.path("micro_irn_varname.xls"))
}

In [7]:
# Define a vector of package names to be used in the script
package_names_vec <- c(
    "tidyverse",
    "ggtext",
    "showtext",
    "tidymodels",
    "furrr",
    "readxl",
    "tictoc",
    "glmnet"
)

# Define a function to install and load required packages
package_prep <- function(names_vec) {
  # Iterate through each package name in the provided vector
  for (name in names_vec) {
    # Check if the package is already installed
    if (!require(name, character.only = TRUE)) {
      # Install the package using renv if not already installed
      install.packages(name)
    }
  }
}

# Pipe the vector of package names into the package_prep function
package_names_vec |> package_prep()


Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: ggtext

Loading required package: showtext

Loading required package: sysfonts

Loading required package: showtextdb

Loading required package: tidymodels

── Attaching packages ────────────────────────────────────── tidymodels 1.3.0 ──

✔ broom        1.0.7     ✔ rsample      1.3.0
✔ dials        1.4.0     ✔ tune         1.3.0
✔ infer        1.0.8     ✔ workflows    1.2.0
✔ modeldata    1.4.0     ✔ workflowsets 1.1.0
✔ parsnip      1.3.1     ✔ y

# Load the data and clean

In [8]:
# Check if the 'readxl' package is already loaded; if not, load the package
if (!"package:readxl" %in% search()) library(readxl)

# Define a function to load and process a table
# Parameters:
# - name: Name of the table to be assigned in the global environment
# - path: Path to the file to be read
# - argyear: Year to be added as a column in the resulting table
# - reading_func: Function to read the file (e.g., read_csv, read_excel)
load_tbl <- function(name, path, argyear, reading_func) {
  # Read the file, process it, and store it in a variable 'result'
  result <- file.path(path) |>  # Generate the full file path
    reading_func() |>           # Read the file using the provided reading function
    mutate(
      year = argyear,           # Add a 'year' column with the specified year
      ID = row_number()         # Add an 'ID' column with sequential row numbers
    ) |> 
    select(where(~!all(is.na(.x)))) # Select only the columns that are not entirely NA
  
  # Assign the processed table to the specified name in the global environment
  assign(name, result, envir = .GlobalEnv)
}

# Call the 'load_tbl' function to load and process data for the year 2021
load_tbl("gfi_2021_tbl",               # Name of the output table
  file.path(paths$data, "micro_irn.csv"),  # File path
  2021,                               # Year to assign
  read_csv                            # Reading function
)

# Call the 'load_tbl' function to load and process data for the year 2017 (variable labels)
load_tbl("gfi_2017_varlable_tbl",            # Name of the output table
  file.path(paths$data, "micro_irn_varlabel.xls"),  # File path
  2017,                               # Year to assign
  read_excel                          # Reading function
)

# Call the 'load_tbl' function to load and process data for the year 2017 (variable names)
load_tbl("gfi_2017_varname_tbl",            # Name of the output table
  file.path(paths$data, "micro_irn_varname.xls"),  # File path
  2017,                               # Year to assign
  read_excel                          # Reading function
)

Rows: 1005 Columns: 84
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): economy, economycode
dbl (78): wpid_random, wgt, female, age, educ, inc_q, emp_in, account, accou...
lgl  (4): urbanicity_f2f, receive_agriculture, remittances, merchantpay_dig

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [18]:
# Replace spaces, colons, and hyphens in column names of `gfi_2017_varlable_tbl` with underscores.
new_names <- gsub("[ :-]", "_", gfi_2017_varlable_tbl |> names())

# Replace consecutive underscores (if any) with a single underscore in the column names.
new_names2 <- gsub("_+", "_", new_names) |> tolower()

# Rename the columns of `gfi_2017_varlable_tbl` using the cleaned-up column names.
gfi_2017_renamed_tbl <- gfi_2017_varlable_tbl |> rename(
  !!!setNames(
    names(gfi_2017_varlable_tbl),  # Current column names of the table.
    new_names2              # Updated column names after transformations.
  )
)

# Final result: `gfi_2017_renamed_tbl` is the table with updated column names.


In [4]:
core_controls <- c(
  "year",
  "ID",
  "Weight",
  "Respondent_is_female",
  "Respondent_age",
  "Respondent_education_level",
  "Respondent_is_in_the_workforce",
  "Within_economy_household_income_quintile",
  "Main_source_of_emergency_funds",
  "Payments_utility_bills",
  "Has_an_account",
  "Has_an_account_at_a_financial_institution",
  "Has_a_mobile_money_account"
)
control_cols2017 <- c(
  core_controls,
  "Payments_wage_payments",
  "Payments_government_transfers",
  "Payments_government_pension"
)
control_cols2021 <- c(
  core_controls,
  "Difficulty_of_emergency_funds_in_30_days",
  "Received_a_government_pension_payment",
  "Received_a_government_transfer_payment",
  "Financially_most_worried",
  "Financially_worried_education",
  "Financially_worried_bills",
  "Financially_worried_medical_cost",
  "Financially_worried_old_age",
  "Difficulty_of_emergency_funds_in_7_days",
  "Paid_a_utility_bill",
  "Received_a_government_pension",
  "Received_a_wage_payment"
)

In [ ]:
long_gfi_2017_tbl <- gfi_2017_renamed_tbl |> 
  # Start with the renamed dataset and apply transformations
  
  # Remove unnecessary columns for further analysis
  select(-c(Economy, Economy_Code, Gallup_World_Poll_identifier)) |> 
  
  # Mutate to clean and standardize categorical variables
  mutate(
    Respondent_education_level = case_match(
      Respondent_education_level,
      c("(dk)", "(rf)") ~ NA, # Replace "don't know" or "refused" responses with NA
      .default = Respondent_education_level
    ),
    Main_source_of_emergency_funds = case_match(
      Main_source_of_emergency_funds,
      c("(dk)", "ref") ~ NA, # Similarly, standardize "don't know" or "refused" responses as NA
      .default = Main_source_of_emergency_funds
    )
  ) |> 
  
  # Reshape data from wide to long format
  pivot_longer(
    cols = -c(!!!syms(control_cols2017)), # Keep control variables static and reshape other columns
    names_to = "Question", # Name for reshaped variable columns
    values_to = "Answers" # Name for reshaped value columns
  ) |> 
  
  # Standardize and categorize answers
  mutate(
    Answers = case_match(
      Answers,
      c("yes", "Yes", "Possible") ~ "1", # Group affirmative responses under "1"
      c("no", "No", "Not possible") ~ "2", # Group negative responses under "2"
      c("(DK)", "(dk)") ~ "3", # Code "don't know" as "3"
      c("(ref)", "ref", "(Refused)") ~ "4", # Code "refused" responses as "4"
      .default = as.character(Answers) # Keep other responses as characters
    )
  )  |> 
  
  # Convert "Answers" to a factor for analysis
  mutate(Answers = fct(Answers)) |> 
  
  # Group by ID to prepare for question numbering
  group_by(ID) |> 
  mutate(Question_no = row_number()) |> # Assign a unique number to each question within each group
  ungroup() |> 
  
  # Apply transformations across multiple columns
  mutate(
    across(
      -c( # Exclude numeric or critical variables from factor conversion
        Respondent_age,
        Weight,
        Question_no,
        ID
      ),
      ~as_factor(.x) # Convert remaining variables to factors
    )
  ) |>
  
  # Additional cleaning and standardization
  mutate(
    Respondent_age = case_when(
      is.na(Respondent_age) ~ NA, # Keep missing values as NA
      Respondent_age == "99+" ~ 100, # Convert "99+" to a numeric value
      .default = as.integer(Respondent_age) # Convert other values to integers
    ),
    Has_an_account_at_a_financial_institution = case_match(
      Has_an_account_at_a_financial_institution,
      "yes" ~ "Yes", # Standardize affirmative responses
      "0" ~ "No", # Standardize negative responses
      .default = Has_an_account_at_a_financial_institution
    ),
    Has_an_account = case_match(
      Has_an_account,
      "yes" ~ "Yes", # Same standardization for another variable
      "0" ~ "No",
      .default = Has_an_account
    ),
    Has_a_mobile_money_account = case_match(
      Has_a_mobile_money_account,
      "yes" ~ "Yes", # Standardize affirmative responses
      "0" ~ "No",
      .default = Has_a_mobile_money_account
    )
  )


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Respondent_age = case_when(...)`.
Caused by warning in `vec_case_when()`:
! NAs introduced by coercion”


In [6]:
# Define the named vector with keys in double quotes (replaced to become similar with 2017 column names.)
names_dict_2021 <- c(
  "saved" = "Saved_in_the_past_year",
"borrowed" = "Borrowed_in_the_past_year",
"receive_wages" = "Received_a_wage_payment",
"receive_transfers" = "Received_a_government_transfer_payment",
"receive_pension" = "Received_a_government_pension_payment",
"pay_utilities" = "Payments_utility_bills",
"mobileowner" = "Owns_a_mobile_phone",
"internetaccess" = "Internet_access",
"anydigpayment" = "Made_or_received_a_digital_payment",
"fin34a" = "Received_wage_payments_into_an_account",
"fin34b" = "Received_wage_payments_to_a_mobile_phone",
"fin34d" = "Received_wage_payments_in_cash",
"fin34e" = "Received_wage_payments_to_a_card",
"fin37" = "Payments_government_transfers",
"fin38" = "Received_a_government_pension",
"fin39a" = "Received_a_government_transfer_or_pension_into_an_account",
"fin39b" = "Received_a_government_transfer_or_pension_to_a_mobile_phone",
"fin39d" = "Received_a_government_transfer_or_pension_in_cash",
"fin39e" = "Received_a_government_transfer_or_pension_to_a_card",
"fin44a" = "Financially_worried_old_age",
"fin44b" = "Financially_worried_medical_cost",
"fin44c" = "Financially_worried_bills",
"fin44d" = "Financially_worried_education",
"fin45" = "Financially_most_worried",
"fin16" = "Saved_for_old_age",
"fin17a" = "Saved_using_an_account_at_a_financial_institution",
"fin17a1" = "Saved_using_a_mobile_money_account",
"fin20" = "Borrowed_for_medical_purposes",
"fin22a" = "Borrowed_from_a_financial_institution",
"fin22b" = "Borrowed_from_family_or_friends",
"fin24" = "Main_source_of_emergency_funds",
"fin24a" = "Difficulty_of_emergency_funds_in_30_days",
"fin24b" = "Difficulty_of_emergency_funds_in_7_days",
"fin30" = "Paid_a_utility_bill",
"fin31a" = "Paid_a_utility_bill_using_an_account",
"fin31b" = "Paid_a_utility_bill_using_a_mobile_phone",
"fin31c" = "Paid_a_utility_bill_in_cash",
"fin32" = "Payments_wage_payments",
"fin33" = "Received_public_sector_wage_payments",
"fin11b" = "Reason_for_no_account_too_expensive",
"fin11c" = "Reason_for_no_account_lack_documentation",
"fin11d" = "Reason_for_no_account_lack_trust",
"fin11e" = "Reason_for_no_account_religious_reasons",
"fin11f" = "Reason_for_no_account_lack_money",
"fin11g" = "Reason_for_no_account_family_member_already_has_one",
"fin11h" = "Reason_for_no_account_no_need_for_financial_services",
"fin13a" = "Use_mobile_money_account_two_or_more_times_a_month",
"fin13b" = "Use_mobile_money_account_to_store_money",
"fin13c" = "Use_mobile_money_account_to_borrow_money",
"fin13d" = "Use_mobile_money_account_without_help",
"fin14_1" = "Use_mobile_phone_to_pay_for_a_purchase_in_store",
"fin14a" = "Made_bill_payments_online_using_the_Internet",
"fin14a1" = "Send_money_to_a_relative_or_friend_online_using_the_Internet",
"fin14b" = "Bought_something_online_using_the_Internet",
"account_mob" = "Has_a_mobile_money_account",
"fin2" = "Has_a_debit_card",
"fin4" = "Used_a_debit_card",
"fin5" = "Used_a_mobile_phone_or_internet_to_access_account",
"fin6" = "Used_a_mobile_phone_or_internet_to_check_account_balance",
"fin7" = "Has_a_credit_card",
"fin8" = "Used_a_credit_card",
"fin8b" = "Paid_credit_card_balances_in_full",
"fin9" = "Made_any_deposit_into_the_account",
"fin9a" = "Make_deposits_into_the_account_two_or_more_times_per_month",
"fin10" = "Withdrew_from_the_account",
"fin10a" = "Withdrew_from_the_account_two_or_more_times_per_month",
"fin10b" = "Used_account_to_store_money",
"fin11_1" = "Unbanked_use_account_without_help",
"fin11a" = "Reason_for_no_account_too_far",
"economy" = "Economy",
"economycode" = "Economy_Code",
"wpid_random" = "Gallup_World_Poll_identifier",
"wgt" = "Weight",
"female" = "Respondent_is_female",
"age" = "Respondent_age",
"educ" = "Respondent_education_level",
"inc_q" = "Within_economy_household_income_quintile",
"emp_in" = "Respondent_is_in_the_workforce",
"account" = "Has_an_account",
"account_fin" = "Has_an_account_at_a_financial_institution",
"year" = "year",
"ID" = "ID"
)

In [ ]:
# Rename columns in gfi_2021_tbl using a names dictionary (names_dict_2021).
# The names are dynamically replaced using setNames and !!! operator for splicing.
long_gfi_2021_tbl <- gfi_2021_tbl |> 
  rename(
    !!!setNames(
      names(gfi_2021_tbl), 
      names_dict_2021[names(gfi_2021_tbl)]
    )
  ) |> 
  # Remove unnecessary columns: Economy, Economy_Code, and Gallup_World_Poll_identifier.
  select(-c(Economy, Economy_Code, Gallup_World_Poll_identifier)) |> 
  # Convert the table to a long format for better analysis.
  # Exclude control columns from being pivoted.
  pivot_longer(
    cols = -c(
      !!!syms(control_cols2021) # Dynamically select control columns to exclude from pivoting.
    ),
    names_to = "Question",      # New column name for the variable names being pivoted.
    values_to = "Answers"       # New column name for the values corresponding to variable names.
  ) |> 
  # Mutate to clean and standardize categorical variables using case_match.
  mutate(
    # Map numeric values of Respondent_is_female to meaningful labels.
    Respondent_is_female = case_match(
      Respondent_is_female,
      1 ~ "Female",
      2 ~ "Male",
      .default = NA
    ),
    # Map education level codes to descriptive labels.
    Respondent_education_level = case_match(
      Respondent_education_level,
      1 ~ "completed primary or less",
      2 ~ "secondary",
      3 ~ "completed tertiary or more",
      .default = NA
    ),
    # Map workforce participation status to descriptive labels.
    Respondent_is_in_the_workforce = case_match(
      Respondent_is_in_the_workforce,
      1 ~ "in workforce",
      2 ~ "out of workforce",
      .default = NA
    ),
    # Map income quintiles to descriptive labels for within-economy household income.
    Within_economy_household_income_quintile = case_match(
      Within_economy_household_income_quintile,
      1 ~ "Poorest 20%",
      2 ~ "Second 20%",
      3 ~ "Middle 20%",
      4 ~ "Fourth 20%",
      5 ~ "Richest 20%",
      .default = NA
    ),
    # Map binary responses for financial institution account ownership.
    Has_an_account_at_a_financial_institution = case_match(
      Has_an_account_at_a_financial_institution,
      1 ~ "Yes",
      0 ~ "No",
      .default = NA
    ),
    # Map binary responses for account ownership in general.
    Has_an_account = case_match(
      Has_an_account,
      1 ~ "Yes",
      0 ~ "No",
      .default = NA
    ),
    # Map binary responses for mobile money account ownership.
    Has_a_mobile_money_account = case_match(
      Has_a_mobile_money_account,
      1 ~ "Yes",
      0 ~ "No",
      .default = NA
    )
  ) |> 
  # Group by respondent ID to create a unique question number for each response.
  group_by(ID) |> 
  mutate(
    Question_no = row_number() # Assign sequential question numbers within each group.
  ) |> 
  ungroup() |> 
  # Convert all columns, except specified ones, to factors for easier analysis.
  mutate(
    across(
      -c(ID, Respondent_age, Weight, Question_no), # Exclude columns that should not be converted.
      ~as_factor(.x) # Convert to factor type.
    )
  )


In [ ]:
# Combine two data tables (long_gfi_2017_tbl and long_gfi_2021_tbl) into one
combined_tbl <- bind_rows(
  long_gfi_2017_tbl,   # Data table for 2017
  long_gfi_2021_tbl    # Data table for 2021
)

# Process the combined data table to calculate weighted sampling
weighted_sum <- combined_tbl |> 
  select(-c(Question, Answers, Question_no, ID)) |> # Remove specific columns to focus on relevant data
  mutate(weighted_s = Weight / sum(Weight)) |>      # Create a column with weights normalized to sum to 1
  slice_sample(                                     # Perform weighted random sampling on the dataset
    n = 1e6,                                       # Number of samples to draw (1 million)
    weight_by = weighted_s,                        # Weights column used for sampling probabilities
    replace = TRUE                                 # Allow sampling with replacement
  )


In [ ]:
combined_tbl

Weight,Respondent_is_female,Respondent_age,Respondent_education_level,Within_economy_household_income_quintile,Respondent_is_in_the_workforce,Main_source_of_emergency_funds,Has_an_account_at_a_financial_institution,Has_a_mobile_money_account,Has_an_account,⋯,Paid_a_utility_bill,Received_a_government_pension,Financially_worried_old_age,Financially_worried_medical_cost,Financially_worried_bills,Financially_worried_education,Financially_most_worried,Received_a_wage_payment,Received_a_government_transfer_payment,Received_a_government_pension_payment
<dbl>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2.171185,Male,19,secondary,Second 20%,out of workforce,Main source: Savings,No,Yes,Yes,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Exploratory Data Analysis and Plots

In [ ]:
# Begin with the `weighted_sum` dataset and pipe it through a series of transformations
plot <- weighted_sum |> 
  # Select relevant columns for analysis
  select(Respondent_age, Respondent_education_level, Respondent_is_female, year) |> 
  # Filter out rows where education level is missing
  filter(!is.na(Respondent_education_level)) |> 
  # Group data by the year column
  group_by(year) |> 
  # Add a new column for the average age within each year group
  mutate(avg_age = mean(Respondent_age, na.rm = TRUE)) |> 
  # Remove grouping structure for further transformations
  ungroup() |> 
  # Modify factor levels to establish specific ordering for certain columns
  mutate(
    # Reorder the `year` column as a factor with a specific sequence
    year = fct_relevel(year, "2017", "2021"),
    # Reorder the `Respondent_is_female` column as a factor (Male first, Female second)
    Respondent_is_female = fct_relevel(Respondent_is_female, "Male", "Female"),
    # Reorder education levels to follow an ascending education hierarchy
    Respondent_education_level = fct_relevel(
      Respondent_education_level,
      "completed primary or less", "secondary", "completed tertiary or more"
    )
  ) |> 
  # Create a boxplot to visualize age distribution across demographic groups
  ggplot(
    aes(
      # Y-axis: Interaction of gender, year, and education level (formatted with separators)
      y = interaction(Respondent_is_female, year, Respondent_education_level, sep = " - "),
      # X-axis: Respondent age
      x = Respondent_age, 
      # Fill: Color boxes based on respondent gender
      fill = Respondent_is_female
    )
  ) +
  # Add boxplots with custom size, width, and no outlier shapes
  geom_boxplot(size = 0.5, width = 0.75, outlier.shape = NA) +
  # Apply a grayscale color palette for the fill aesthetic
  scale_fill_grey(
    start = 1, end = 0.8, 
    name = "Respondent's Gender", 
    labels = c("Male", "Female")
  ) +
  # Customize the legend to use plain square shapes and specific sizes/colors
  guides(fill = guide_legend(
    override.aes = list(
      shape = 22, size = 5, fill = c("grey80", "grey50"), color = "black", linetype = 0
    )
  )) +
  # Add plot labels including title, subtitle, axis titles, and a data source caption
  labs(
    title = "Age Distribution by Education Level, Gender, and Year",
    subtitle = "Based on World Bank's Global Financial Inclusion Survey (Iran, 2017 & 2021)",
    y = "Demographic Groups (by Education, Year, and Gender)",
    x = "Respondent Age",
    fill = "Gender",
    caption = "Source: World Bank Global Financial Inclusion Survey"
  ) +
  # Use a minimalistic theme with a base font size
  theme_minimal(base_size = 12) +
  # Further customize the plot's appearance
  theme(
    legend.position = "right", # Place legend to the right
    axis.title.y = element_text(face = "bold"), # Bold font for Y-axis title
    axis.title.x = element_text(face = "bold"), # Bold font for X-axis title
    axis.text.y = element_text(face = "bold"), # Bold font for Y-axis labels
    panel.grid.major.y = element_line(color = "gray90", linewidth = 0.3), # Subtle horizontal gridlines
    panel.grid.minor = element_blank(), # Remove minor gridlines
    axis.ticks = element_line(color = "gray60"), # Subtle axis ticks
    axis.text = element_text(size = 10), # Adjust axis text size
    text = element_text(family = "Times New Roman"), # Use "Times New Roman" font for all text
    panel.border = element_rect(color = "black", fill = NA, linewidth = 1) # Add a black border around the panel
  )
# Save the plot with custom dimensions
file_name <- "age_distribution_plot.png"
ggsave(
  filename = file_name,
  plot = plot,
  width = 12, # Customize width
  height = 8, # Customize height
  dpi = 300   # High resolution
)

# Display the saved image in the notebook
img <- png::readPNG(file_name)
grid::grid.raster(img)

In [ ]:
# Prepare the data for visualization
plot_data <- weighted_sum |> 
  # Select relevant columns
  select(
    Respondent_education_level,
    Respondent_is_in_the_workforce,
    Within_economy_household_income_quintile,
    Has_a_mobile_money_account,
    Has_an_account_at_a_financial_institution,
    year,
    Respondent_is_female
  ) |> 
  # Modify levels for categorical variables for better ordering in the plot
  mutate(
    Respondent_education_level = fct_relevel(
      Respondent_education_level,
      "completed primary or less", "secondary", "completed tertiary or more"
    ),
    Respondent_is_in_the_workforce = fct_relevel(
      Respondent_is_in_the_workforce, "out of workforce", "in workforce"
    ),
    Within_economy_household_income_quintile = fct_relevel(
      Within_economy_household_income_quintile,
      "Poorest 20%", "Second 20%", "Middle 20%", "Fourth 20%", "Richest 20%"
    ),
    # Create a combined category for demographic groups
    category_combination = interaction(
      Respondent_is_in_the_workforce,
      Respondent_education_level,
      Within_economy_household_income_quintile,
      sep = " - "
    )
  ) |> 
  # Remove rows with missing values for key columns
  filter(
    !is.na(Has_a_mobile_money_account),
    !is.na(Has_an_account_at_a_financial_institution),
    !is.na(category_combination)
  ) |> 
  # Group the data by year, demographic group, and gender
  group_by(year, category_combination, Respondent_is_female) |> 
  # Calculate the percentage of "Yes" responses for mobile money and financial account ownership
  summarise(
    mobile_money_yes_pct = mean(Has_a_mobile_money_account == "Yes", na.rm = TRUE) * 100,
    financial_account_yes_pct = mean(Has_an_account_at_a_financial_institution == "Yes", na.rm = TRUE) * 100,
    .groups = "drop"  # Drop grouping after summarization
  ) |> 
  # Reshape data from wide to long format for easier plotting
  pivot_longer(
    cols = c(mobile_money_yes_pct, financial_account_yes_pct),
    names_to = "account_type",
    values_to = "yes_pct"
  )

# Create separate datasets for males and females
male_data <- plot_data |> filter(Respondent_is_female == "Male")
female_data <- plot_data |> filter(Respondent_is_female == "Female")

# Combine male and female data to calculate gender gaps (for plotting connectors)
connector_data <- male_data |> 
  rename(yes_pct_male = yes_pct) |> 
  inner_join(
    female_data |> rename(yes_pct_female = yes_pct),
    by = c("year", "category_combination", "account_type")
  )

# Generate the plot using ggplot2
plot <- ggplot(plot_data, aes(x = category_combination, y = yes_pct)) +
  # Add connector lines between male and female percentages
  geom_segment(
    data = connector_data,
    aes(
      x = category_combination,
      xend = category_combination,
      y = yes_pct_male,
      yend = yes_pct_female,
      linetype = account_type
    ),
    color = "black"
  ) +
  # Add points for male and female percentages
  geom_point(
    aes(
      shape = interaction(account_type, Respondent_is_female)
    ),
    size = 2,  # Specify point size
    color = "black"
  ) +
  # Create a separate panel for each year
  facet_wrap(~year, nrow = 1) +
  # Flip coordinates to make the chart horizontal
  coord_flip() +
  # Use a minimal theme and customize gridlines
  theme_minimal() +
  theme(
    panel.grid.major.y = element_blank(),  # Remove horizontal gridlines
    panel.grid.major.x = element_line(color = "gray90"),  # Retain vertical gridlines
    axis.line = element_line(color = "black"),
    panel.border = element_rect(color = "black", fill = NA)  # Add a border around the plot
  ) +
  # Customize the legend for line types (solid and dashed)
  scale_linetype_manual(
    values = c(
      "mobile_money_yes_pct" = "solid",
      "financial_account_yes_pct" = "dashed"
    ),
    labels = c("Mobile Money", "Financial Account")
  ) +
  # Customize the legend for shapes (distinct for male and female)
  scale_shape_manual(
    values = c(
      "mobile_money_yes_pct.Male" = 19,  # Solid circle
      "financial_account_yes_pct.Male" = 1,  # Hollow circle
      "mobile_money_yes_pct.Female" = 15,  # Solid square
      "financial_account_yes_pct.Female" = 0  # Hollow square
    ),
    labels = c(
      "Mobile Money (Male)",
      "Financial Account (Male)",
      "Mobile Money (Female)",
      "Financial Account (Female)"
    )
  ) +
  # Add plot titles, axis labels, and legend titles
  labs(
    title = "Account Ownership by Demographics in Iran (2017 vs 2021)",
    subtitle = "Data from the World Bank's Financial Inclusion Survey",
    x = "Demographic Groups",
    y = "Percentage of Ownership",
    shape = "Account Type and Gender",
    linetype = "Gender Gap"
  )
# Save the plot with custom dimensions
file_name <- "account_ownership_plot.png"
ggsave(
  filename = file_name,
  plot = plot,
  width = 12, # Customize width
  height = 8, # Customize height
  dpi = 300   # High resolution
)

# Display the saved image in the notebook
img <- png::readPNG(file_name)
grid::grid.raster(img)

In [ ]:
# Select and process relevant columns from the dataset
plot_data <- weighted_sum |> 
  select(
    Respondent_education_level,                   # Education level of respondents
    Has_an_account_at_a_financial_institution,    # Indicator for financial institution account ownership
    Has_a_mobile_money_account,                   # Indicator for mobile money account ownership
    Within_economy_household_income_quintile,     # Income quintile classification
    Respondent_is_in_the_workforce,               # Workforce participation status
    year,                                         # Year of survey
    Respondent_is_female                          # Gender of respondents
  ) |> 
  # Relevel categorical variables for better ordering in visualization
  mutate(
    Respondent_education_level = fct_relevel(
      Respondent_education_level,
      "completed primary or less",
      "secondary",
      "completed tertiary or more"
    ),
    Within_economy_household_income_quintile = fct_relevel(
      Within_economy_household_income_quintile,
      "Poorest 20%", "Second 20%", "Middle 20%", "Fourth 20%", "Richest 20%"
    ),
    Respondent_is_female = fct_relevel(
      Respondent_is_female,
      "Male", "Female"
    ),
    year = fct_relevel(year, "2017", "2021"),
    category_combination = interaction(
      Respondent_education_level,
      Within_economy_household_income_quintile,
      sep = " - "                           # Combine education and income quintile for grouping
    )
  ) |> 
  # Filter rows with missing values in key columns
  filter(
    !is.na(Has_an_account_at_a_financial_institution),
    !is.na(Has_a_mobile_money_account),
    !is.na(category_combination)
  ) |> 
  # Group data by year, demographic category, and gender
  group_by(year, category_combination, Respondent_is_female) |> 
  # Calculate percentages for each account ownership type
  summarise(
    mobile_money_only_pct = mean(
      Respondent_is_in_the_workforce == "in workforce" & 
      Has_a_mobile_money_account == "Yes" & 
      Has_an_account_at_a_financial_institution == "No", 
      na.rm = TRUE
    ) * 100,
    financial_account_only_pct = mean(
      Respondent_is_in_the_workforce == "in workforce" & 
      Has_a_mobile_money_account == "No" & 
      Has_an_account_at_a_financial_institution == "Yes", 
      na.rm = TRUE
    ) * 100,
    both_accounts_pct = mean(
      Respondent_is_in_the_workforce == "in workforce" & 
      Has_a_mobile_money_account == "Yes" & 
      Has_an_account_at_a_financial_institution == "Yes", 
      na.rm = TRUE
    ) * 100,
    neither_accounts_pct = mean(
      Respondent_is_in_the_workforce == "in workforce" & 
      Has_a_mobile_money_account == "No" & 
      Has_an_account_at_a_financial_institution == "No", 
      na.rm = TRUE
    ) * 100,
    .groups = "drop"                         # Drop grouping after summarization
  ) |> 
  # Transform data into a long format for plotting
  pivot_longer(
    cols = c(mobile_money_only_pct, financial_account_only_pct, both_accounts_pct, neither_accounts_pct),
    names_to = "participation_type",          # New column for account type
    values_to = "yes_pct"                     # New column for percentage values
  ) |> 
  # Create labels for account ownership types
  mutate(
    participation_label = factor(
      participation_type,
      levels = c("both_accounts_pct", "financial_account_only_pct", "mobile_money_only_pct", "neither_accounts_pct"),
      labels = c("With Both Accounts", "With Financial Account Only", "With Mobile Money Only", "With No Account")
    ),
    gender_year = paste(Respondent_is_female, year, sep = " ") # Combine gender and year for labeling
  )

# Prepare data for connector lines in the plot
connector_data <- plot_data |> 
  pivot_wider(
    id_cols = c(year, category_combination, participation_type, participation_label),
    names_from = Respondent_is_female,       # Create separate columns for Male and Female percentages
    values_from = yes_pct                    # Values are percentages for each gender
  )

# Plotting using ggplot2
plot <- ggplot(plot_data, aes(x = category_combination, y = yes_pct)) +
  # Add connector lines for 2021 data
  geom_segment(
    data = connector_data |> filter(year == 2021),
    aes(
      x = category_combination,
      xend = category_combination,
      y = Male,
      yend = Female,
      size = factor(year),                   # Line size by year
      color = factor(year)                   # Line color by year
    ),
    show.legend = F                          # Remove legend for connectors
  ) +
  # Add connector lines for 2017 data
  geom_segment(
    data = connector_data |> filter(year == 2017),
    aes(
      x = category_combination,
      xend = category_combination,
      y = Male,
      yend = Female,
      size = factor(year),
      color = factor(year)
    ),
    show.legend = F
  ) +
  # Add points representing percentages
  geom_point(
    aes(
      shape = factor(gender_year)            # Shape varies by gender and year
    ),
    size = 2,                                # Point size
    color = "black"                          # Point color
  ) +
  # Facet into separate panels for account types
  facet_wrap(~ participation_label, nrow = 1) +
  coord_flip() +                             # Flip coordinates for horizontal chart
  theme_minimal() +                          # Minimal theme
  theme(
    panel.grid.major.y = element_blank(),    # Remove horizontal gridlines
    panel.grid.major.x = element_line(color = "gray90"),
    axis.line = element_line(color = "black"),
    panel.border = element_rect(color = "black", fill = NA),
    legend.position = "right"                # Position legend on the right
  ) +
  # Adjust line size and color scales
  scale_size_manual(
    values = c("2017" = 1, "2021" = 3),      # Thicker lines for 2021
    guide = 'none'
  ) +
  scale_color_manual(
    values = c("2017" = "black", "2021" = "lightgray"),
    guide = 'none'
  ) +
  # Customize legend for shape aesthetics
  scale_shape_manual(
    values = c(
      "Male 2017" = 20,
      "Female 2017" = 15,
      "Male 2021" = 1,
      "Female 2021" = 0
    ),
    labels = c(
      "Female (2017)",
      "Female (2021)",
      "Male (2017)",
      "Male (2021)"
    )
  ) +
  # Add titles and labels
  labs(
    title = "Labor Force Participation Gender Gaps by Financial Inclusion Leading Variables (2017 & 2021)",
    subtitle = "Data from the World Bank's Global Financial Inclusion Survey (Iran)",
    x = "Demographic Groups (by Education and Income)",
    y = "Percentage of Labor Force Participants",
    shape = "Gender and Year",
    caption = "Source: World Bank Global Financial Inclusion Survey"
  )
# Save the plot with custom dimensions
file_name <- "lfp_gap_plot.png"
ggsave(
  filename = file_name,
  plot = plot,
  width = 12, # Customize width
  height = 8, # Customize height
  dpi = 300   # High resolution
)

# Display the saved image in the notebook
img <- png::readPNG(file_name)
grid::grid.raster(img)

# Wrangle and Models

## Paremtric without Bootsrtap

In [ ]:
weighted_sumw <- combined_tbl |> 
  mutate(
    Within_economy_household_income_quintile  = factor(Within_economy_household_income_quintile, levels = c("Poorest 20%","Second 20%","Middle 20%","Fourth 20%","Richest 20%"),ordered = T),
    Respondent_education_level = factor(Respondent_education_level, levels = c("completed primary or less","secondary","completed tertiary or more"),ordered = T)
)
 workflow() |> 
  add_model(
    logistic_reg() |> set_engine("glm") |> set_mode("classification")
  ) |> 
  add_recipe(
    recipe(Respondent_is_in_the_workforce ~ 
      Respondent_is_female + 
      Respondent_age+
      Has_an_account_at_a_financial_institution+
      Has_a_mobile_money_account+
      Within_economy_household_income_quintile+
           year+
      Respondent_education_level,
     data = weighted_sumw,
          weights = weight) |> 
      step_dummy(c(year,Respondent_is_female,Has_an_account_at_a_financial_institution,Has_a_mobile_money_account)) |> 
      step_ordinalscore(c(Within_economy_household_income_quintile,Respondent_education_level)) |> 
      step_interact(~.:starts_with("year"))
  ) |> 
fit(data = weighted_sumw) |> 
   extract_fit_parsnip() |> 
   tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.630387769,5.791085e-02,-1.088549e+01,1.351747e-27
Respondent_age,-0.007826444,6.264152e-04,-1.249402e+01,8.048189e-36
Within_economy_household_income_quintile,-0.030123903,6.809100e-03,-4.424065e+00,9.686082e-06
Respondent_education_level,0.559857790,1.479642e-02,3.783738e+01,0.000000e+00
year_X2021,-18.935680750,2.744874e+02,-6.898561e-02,9.450011e-01
Respondent_is_female_Female,-1.696339211,1.908269e-02,-8.889412e+01,0.000000e+00
Has_an_account_at_a_financial_institution_Yes,0.749857829,4.148945e-02,1.807346e+01,5.157652e-73
Has_a_mobile_money_account_Yes,0.461204722,2.173457e-02,2.121987e+01,6.259683e-100
Respondent_age_x_year_X2021,0.007826444,3.088155e+00,2.534343e-03,9.979779e-01


## Parametric with Bootsrap

In [ ]:
# Generate bootstraps
boot_samples <- bootstraps(weighted_sumw, times = 2.5e2)
fit_model <- function(split) {
  boot_data <- analysis(split)
    workflow() |> 
    add_model(
      logistic_reg() |> set_engine("glm") |> set_mode("classification")
    ) |> 
    add_recipe(
      recipe(
          Respondent_is_in_the_workforce ~ Respondent_is_female + Respondent_age +Has_an_account_at_a_financial_institution +
          Has_a_mobile_money_account + Within_economy_household_income_quintile + year + Respondent_education_level,
       data = boot_data) |> 
        step_dummy(year) |> 
        step_ordinalscore(Within_economy_household_income_quintile) |> 
        step_ordinalscore(Respondent_education_level)
    ) |> 
    fit(data = boot_data) |> 
    extract_fit_parsnip() |> 
    tidy()
}

# Fit models on bootstrap samples with progress bar
bootstrap_results_tbl <- map(
  boot_samples$splits,
  fit_model,
  .progress = list(# Display progress bar
    type = "iterator",
    format = "Boostrapping Iterations: {cli::pb_bar} {cli::pb_percent}",
    clear = TRUE
  )
) |> 
bind_rows(.id = "bootstrap_id")

# View or save the results
bootstrap_summary  <- bootstrap_results_tbl|> 
  group_by(term) |> 
  summarize(
    mean_estimate = mean(estimate),
    median_estimate = median(estimate),
    lower_ci = quantile(estimate, 0.025),
    upper_ci = quantile(estimate, 0.975)
  )
bootstrap_summary


In [ ]:
# Updated plot with aesthetic changes
plot <- bootstrap_results_tbl |> 
  ggplot(aes(x = estimate)) +
  # Changed the fill to none and added border to silhouette
  geom_histogram(bins = 50, fill = "gray80", size = 0.3) +
  # Vertical lines for mean and confidence intervals
  geom_vline(data = bootstrap_summary, aes(xintercept = mean_estimate), linetype = "longdash", size = 0.5) +
  geom_vline(data = bootstrap_summary, aes(xintercept = lower_ci), linetype = "dotted", size = 0.5) +
  geom_vline(data = bootstrap_summary, aes(xintercept = upper_ci), linetype = "dotted", size = 0.5) +
  # Adding text for the mean, lower, and upper CI with refined positioning
  geom_text(data = bootstrap_summary, aes(x = mean_estimate, y = 2.5, label = sprintf("Mean %.2f", mean_estimate)),
            vjust = 0, hjust = 0.5, size = 3) +
  geom_text(data = bootstrap_summary, aes(x = lower_ci, y = Inf, label = sprintf("2.5%%[%.2f]", lower_ci)),
            vjust = 2.5, hjust = 0.5, size = 3) +
  geom_text(data = bootstrap_summary, aes(x = upper_ci, y = Inf, label = sprintf("97.5%%[%.2f]", upper_ci)),
            vjust = 2.5, hjust = 0.5, size = 3) +
  # Facet labels with more subtle background and adjusted text size
  facet_wrap(~term, scales = "free", ncol = 3, labeller = as_labeller(c(
    "(Intercept)" = "Intercept",
    "Has_a_mobile_money_accountYes" = "Having a Mobile Money Account",
    "Has_an_account_at_a_financial_institutionYes" = "Having an Account at\n a Financial Institution",
    "Respondent_age" = "Age",
    "Respondent_education_level" = "Education Level",
    "Respondent_is_femaleFemale" = "Being a Female",
    "Within_economy_household_income_quintile" = "Household Income Quantile",
    "year_X2021" = "Year"
  ))) +
  theme_bw(base_size = 12) +
  theme(
    strip.background = element_rect(fill = "white", color = "white",size = 0.25), # Faded borders for facet
    strip.text = element_text(size = 12, face = "bold", family = "Times New Roman"),
    text = element_text(size = 12, family = "Times New Roman"),
    plot.title = element_text(size = 20, face = "bold", hjust = 0.5, family = "Times New Roman"),
    plot.subtitle = element_text(size = 16, face = "italic", hjust = 0.5, family = "Times New Roman"),
    axis.title = element_text(size = 12, family = "Times New Roman"),
    axis.text = element_text(size = 10, family = "Times New Roman"),
    legend.position = "bottom",
    legend.title = element_blank(),
    panel.grid.major = element_blank(), # Remove grid lines
    panel.grid.minor = element_blank(),
    plot.margin = margin(20, 20, 20, 20) # Add some padding around the plot
  ) +
  labs(
    title = "Distributions of Bootstrap Coefficients",
    subtitle = "World Bank Financial Inclusion Survey: Iran, 2017 and 2021",
    x = "Coefficient Estimate",
    y = "Frequency",
    fill = "Count"
  )

# Save the updated plot with the new style
file_name <- "bootstrap_density_plot.png"
ggsave(
  filename = file_name,
  plot = plot,
  width = 12,
  height = 8,
  dpi = 300
)

# Display the plot within your notebook
img <- png::readPNG(file_name)
grid::grid.raster(img)


In [ ]:
fit_model_2 <- function(split) {
  boot_data <- analysis(split)
  workflow() |> 
    add_model(
      logistic_reg() |> set_engine("glm") |> set_mode("classification")
    ) |> 
    add_recipe(
    recipe(Respondent_is_in_the_workforce ~ 
      Respondent_is_female + 
      Respondent_age+
      Has_an_account_at_a_financial_institution+
      Has_a_mobile_money_account+
      Within_economy_household_income_quintile+
           year+
      Respondent_education_level,
     data = weighted_sumw,
          weights = weight) |> 
      step_dummy(c(year,Respondent_is_female,Has_an_account_at_a_financial_institution,Has_a_mobile_money_account)) |> 
      step_ordinalscore(c(Within_economy_household_income_quintile,Respondent_education_level)) |> 
      step_interact(~.:starts_with("year"))
    ) |> 
    fit(data = boot_data) |> 
    extract_fit_parsnip() |> 
    tidy()
}

# Fit models on bootstrap samples with progress bar
bootstrap_results_2_tbl <- map(
  boot_samples$splits,
  fit_model_2,
  .progress = list(
    type = "iterator",
    format = "Bootstrapping Iterations: {cli::pb_bar} {cli::pb_percent}",
    clear = TRUE
  )
) |> 
bind_rows(.id = "bootstrap_id")

# Summarize bootstrap results
bootstrap_summary_2 <- bootstrap_results_2_tbl |> 
  group_by(term) |> 
  summarize(
    mean_estimate = mean(estimate),
    median_estimate = median(estimate),
    lower_ci = quantile(estimate, 0.025),
    upper_ci = quantile(estimate, 0.975)
  )

bootstrap_summary_2


In [ ]:
# Updated plot with aesthetic changes
plot <- bootstrap_results_2_tbl |> 
  ggplot(aes(x = estimate)) +
  # Changed the fill to none and added border to silhouette
  geom_histogram(bins = 50, fill = "gray80", size = 0.3) +
  # Vertical lines for mean and confidence intervals
  geom_vline(data = bootstrap_summary_2, aes(xintercept = mean_estimate), linetype = "longdash", size = 0.5) +
  geom_vline(data = bootstrap_summary_2, aes(xintercept = lower_ci), linetype = "dotted", size = 0.5) +
  geom_vline(data = bootstrap_summary_2, aes(xintercept = upper_ci), linetype = "dotted", size = 0.5) +
  # Adding text for the mean, lower, and upper CI with refined positioning
  geom_text(data = bootstrap_summary_2, aes(x = mean_estimate, y = 2.5, label = sprintf("Mean %.2f", mean_estimate)),
            vjust = 0, hjust = 0.5, size = 3) +
  geom_text(data = bootstrap_summary_2, aes(x = lower_ci, y = Inf, label = sprintf("2.5%%[%.2f]", lower_ci)),
            vjust = 2.5, hjust = 0.5, size = 3) +
  geom_text(data = bootstrap_summary_2, aes(x = upper_ci, y = Inf, label = sprintf("97.5%%[%.2f]", upper_ci)),
            vjust = 2.5, hjust = 0.5, size = 3) +
  # Facet labels with more subtle background and adjusted text size
  facet_wrap(~term, scales = "free", ncol = 3, labeller = as_labeller(c(
    "(Intercept)" = "Intercept",
    "Has_a_mobile_money_account_Yes" = "Having a Mobile Money Account",
    "Has_an_account_at_a_financial_institution_Yes" = "Having an Account at\n a Financial Institution",
    "Respondent_age" = "Age",
    "Respondent_education_level" = "Education Level",
    "Respondent_is_female_Female" = "Being a Female",
    "Within_economy_household_income_quintile" = "Household Income Quantile",
    "year_X2021" = "Year",
    "Respondent_age_x_year_X2021" = "Age x Year 2021",
    "Respondent_education_level_x_year_X2021" = "Education Level x Year 2021",
    "Within_economy_household_income_quintile_x_year_X2021" = "Income Quantile x Year 2021",
    "`Respondent_is_in_the_workforcein workforce_x_year_X2021`" = "Workforce Status x Year 2021",
    "year_X2021_x_Has_a_mobile_money_account_Yes" = "Mobile Money Account x Year 2021",
    "year_X2021_x_Has_an_account_at_a_financial_institution_Yes" = "Account at Financial Institution x Year 2021",
    "year_X2021_x_Respondent_is_female_Female" = "Female x Year 2021"
  ))) +
  theme_bw(base_size = 12) +
  theme(
    strip.background = element_rect(fill = "white", color = "white",size = 0.25), # Faded borders for facet
    strip.text = element_text(size = 12, face = "bold", family = "Times New Roman"),
    text = element_text(size = 12, family = "Times New Roman"),
    plot.title = element_text(size = 20, face = "bold", hjust = 0.5, family = "Times New Roman"),
    plot.subtitle = element_text(size = 16, face = "italic", hjust = 0.5, family = "Times New Roman"),
    axis.title = element_text(size = 12, family = "Times New Roman"),
    axis.text = element_text(size = 10, family = "Times New Roman"),
    legend.position = "bottom",
    legend.title = element_blank(),
    panel.grid.major = element_blank(), # Remove grid lines
    panel.grid.minor = element_blank(),
    plot.margin = margin(20, 20, 20, 20) # Add some padding around the plot
  ) +
  labs(
title = "Distributions of Bootstrap Coefficients",
subtitle = "Including Slopes Interacted with Year",
    x = "Coefficient Estimate",
    y = "Frequency",
    fill = "Count"
  )

# Save the updated plot with the new style
file_name <- "bootstrap_density_2_plot.png"
ggsave(
  filename = file_name,
  plot = plot,
  width = 12,
  height = 8,
  dpi = 300
)

# Display the plot within your notebook
img <- png::readPNG(file_name)
grid::grid.raster(img)

